Copied from this source:
https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html and Sophie's tutorial.

In [0]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            # tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            # tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [0]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

In [0]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [0]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [5]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,857,261 trainable parameters


In [6]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

print(PAD_IDX)

1


*** Start reading new stuff here***
To speed up process, I am only running one epoch currently. Change it back to at least 10 when you are done.

In [7]:
import math
import time


def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 0m 29s
	Train Loss: 5.682 | Train PPL: 293.448
	 Val. Loss: 5.251 |  Val. PPL: 190.777
| Test Loss: 5.259 | Test PPL: 192.284 |


How could you use this model to predict a new value? I think like this?

In [9]:
x_test = next(enumerate(train_iterator))[1].src.to(device)
y_test = torch.tensor([[0]]).to(device) #next(enumerate(train_iter))[1].src.to(device)  # i think maybe we can leave the target empty?
result = model(x_test, y_test)
print(result)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')


How could we translate this back? Well, let's explore how to create examples from scratch.

In [10]:
from torchtext.data import Example
test = Example.fromlist(data=['zwei junge weiße männer sind i m freien in der nähe vieler büsche'], fields=[('src', SRC)])
print(test.src)

['zwei', 'junge', 'weiße', 'männer', 'sind', 'i', 'm', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche']


Make this singular sentence into a dataset.

In [11]:
from torchtext.data import Dataset
custom_test_data = Dataset([test], fields=[('src', SRC)])
print(type(custom_test_data))

<class 'torchtext.data.dataset.Dataset'>


And make a bucket. Then we'll be able to view the vector form of this sentence that we created from scratch:

In [13]:
 from torchtext.data import BucketIterator
 test_iter = BucketIterator(dataset=custom_test_data, batch_size=1)
 print(next(enumerate(test_iter))[1].src)

tensor([[   2],
        [  18],
        [  28],
        [ 216],
        [  32],
        [  86],
        [  20],
        [  21],
        [  90],
        [   7],
        [  15],
        [ 116],
        [7663],
        [3219],
        [   3]])


Then, we could apply the model to predict the english translation vector

In [14]:
x_test = next(enumerate(test_iter))[1].src.to(device)
y_test = torch.tensor([[0]]).to(device) #next(enumerate(test_iter))[1].src.to(device)  # I think we can keep this target empty...?
result = model(x_test,y_test)
print(result)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')


How do we convert a tensor back to a sentence? Let's first define this helper function.

In [0]:
def word_ids_to_sentence(id_tensor, vocab, join=None):
    """Converts a sequence of word ids to a sentence"""
    if isinstance(id_tensor, torch.LongTensor):
        ids = id_tensor.transpose(0, 1).contiguous().view(-1)
    elif isinstance(id_tensor, np.ndarray):
        ids = id_tensor.transpose().reshape(-1)
    batch = [vocab.itos[ind] for ind in ids] # denumericalize
    if join is None:
        return batch
    else:
        return 'model translation: '+join.join(batch)

Applying this method doesn't give us an error but also doesn't give us something that makes sense, so we'll need to investigate further:

In [16]:
import numpy as np
print('x-test sample: ' 'zwei junge weiße männer sind i m freien in der nähe vieler büsche')
print('correct translation: ', 'two young white men are outdoors near many bushes')
arrs = model(x_test, y_test).cpu().data.numpy()
word_ids_to_sentence(np.argmax(arrs, axis=2), TRG.vocab, join=' ')


x-test sample: zwei junge weiße männer sind i m freien in der nähe vieler büsche
correct translation:  two young white men are outdoors near many bushes


'model translation: <unk>'

### We can correctly create an english sentence, convert it to a vector, and convert it back to an english sentence:

In [18]:
print('original: ', 'a man in an orange hat starring at something.')
train = Example.fromlist(
    data=['a man in an orange hat starring at something.'], 
    fields=[('trg', TRG)])
custom_train_data = Dataset([train], fields=[('trg', TRG)])
train_iter = BucketIterator(
    dataset=custom_train_data, batch_size=1)
s = next(enumerate(train_iter))[1].trg.to(device)

word_ids_to_sentence(s.cpu().data.numpy(), TRG.vocab, join=' ')

original:  a man in an orange hat starring at something.


'model translation: <sos> a man in an orange hat starring at something . <eos>'

### We can correctly create an german sentence, convert it to a vector, and convert it back to an german sentence:

In [19]:
print('original', 'ein Mann in einem orangefarbenen Hut mit einem Blick auf etwas.')
train = Example.fromlist(
    data=['ein Mann in einem orangefarbenen Hut mit einem Blick auf etwas.'], 
    fields=[('src', SRC)])
custom_train_data = Dataset([train], fields=[('src', SRC)])
train_iter = BucketIterator(
    dataset=custom_train_data, batch_size=1)
s = next(enumerate(train_iter))[1].src.to(device)

word_ids_to_sentence(s.cpu().data.numpy(), SRC.vocab, join=' ')

original ein Mann in einem orangefarbenen Hut mit einem Blick auf etwas.


'model translation: <sos> ein mann in einem orangefarbenen hut mit einem blick auf etwas . <eos>'

For some reason, though, I am having difficulty taking a german sentence from the actual training set and converting it to a vector and back to a proper translation:

In [20]:
data = ' '.join(train_data[0].src)
print(data)
train = Example.fromlist(
    data=data, 
    fields=[('src', SRC)])
custom_train_data = Dataset([train], fields=[('src', SRC)])
train_iter = BucketIterator(
    dataset=custom_train_data, batch_size=1)
s = next(enumerate(train_iter))[1].src.to(device)

word_ids_to_sentence(s.cpu().data.numpy(), SRC.vocab, join=' ')

zwei junge weiße männer sind i m freien in der nähe vieler büsche .


'model translation: <sos> <unk> <eos>'

In addition, I am having difficulty taking an english sentence from the actual training set and converting it to a vector and back to a proper translation:

In [21]:
data = ' '.join(train_data[0].trg)
print(data)
train = Example.fromlist(
    data=data, 
    fields=[('src', SRC)])
custom_train_data = Dataset([train], fields=[('src', SRC)])
train_iter = BucketIterator(
    dataset=custom_train_data, batch_size=1)
s = next(enumerate(train_iter))[1].src.to(device)

word_ids_to_sentence(s.cpu().data.numpy(), SRC.vocab, join=' ')

two young , white males are outside near many bushes .


'model translation: <sos> t <eos>'

Once you figure out the issue, I think you may be just about done.